In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plt

MODEL = "text-curie-001"

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.font_manager as font_manager
import urllib.request


urllib.request.urlretrieve(
    "https://github.com/google/fonts/raw/main/ofl/ibmplexmono/IBMPlexMono-Regular.ttf",
    "IBMPlexMono-Regular.ttf",
)
fe = font_manager.FontEntry(fname="IBMPlexMono-Regular.ttf", name="plexmono")
font_manager.fontManager.ttflist.append(fe)
plt.rcParams.update(
    {
        "axes.facecolor": "#f5f4e9",
        "grid.color": "#AAAAAA",
        "axes.edgecolor": "#333333",
        "figure.facecolor": "#FFFFFF",
        "axes.grid": False,
        "axes.prop_cycle": plt.cycler("color", plt.cm.Dark2.colors),
        "font.family": fe.name,
        "figure.figsize": (3.5, 3.5 / 1.2),
        "ytick.left": True,
        "xtick.bottom": True,
    }
)

import random

np.random.seed(0)
random.seed(0)

In [ ]:
# defining the function. At y=0 to get a 1D cut at the origin
def ackley_1d(x, offset=2.1):
    x = x - offset
    # the formula is rather large
    out = -(
        -20 * np.exp(-0.2 * np.sqrt(0.5 * (x**2)))
        - np.exp(0.5 * (np.cos(2 * np.pi * x) + np.cos(0)))
        + np.e
        + 20
    )

    # returning
    return out


x = np.linspace(-5, 5, 200)
plt.plot(x, ackley_1d(x))

In [ ]:
import bolift
from langchain.prompts.prompt import PromptTemplate

## Few Shot

In [ ]:
def run_experiment(asktell_class, N=10, aq="random"):
    asktell = asktell_class(
        x_formatter=lambda x: f"f({float(x):0.2f}) for the shifted Ackley function",
        model=MODEL,
    )
    pool = bolift.Pool(list(x), formatter=asktell.format_x)
    # start low
    xi = x[np.random.choice(25, 1)[0]]
    pool.choose(xi)
    # give one point
    asktell.tell(xi, ackley_1d(xi))
    point = [(xi, ackley_1d(xi))]
    best = point[0][1]
    for i in range(N):
        if i == N - 1 and aq != "random":
            aq = "greedy"
        px, _, py = asktell.ask(pool, k=1, aq_fxn=aq)
        xc = float(px[0])
        pool.choose(px[0])
        y = ackley_1d(xc)
        asktell.tell(xc, y)
        best = max(y, best)
        point.append((xc, best))
    return point

In [ ]:
N = 10
M = 25
plt.figure(figsize=(3.5, 3.5 / 1.2))
points = []
for i in range(M):
    point = run_experiment(bolift.AskTellFewShotTopk, N=N, aq="random")
    points.append(point)
    plt.plot(range(N + 1), [y for x, y in point], color="C0", alpha=0.1)
# plot mean
points = np.array(points)
plt.plot(range(N + 1), points.mean(axis=0)[:, 1], color="C0", label="Random")


points = []
for i in range(M):
    point = run_experiment(bolift.AskTellFewShotTopk, N=N, aq="greedy")
    points.append(point)
    plt.plot(range(N + 1), [y for x, y in point], color="C2", alpha=0.1)
# plot mean
points = np.array(points)
plt.plot(range(N + 1), points.mean(axis=0)[:, 1], color="C2", label="Greedy")

points = []
for i in range(M):
    point = run_experiment(bolift.AskTellFewShotTopk, N=N, aq="expected_improvement")
    points.append(point)
    plt.plot(range(N + 1), [y for x, y in point], color="C1", alpha=0.1)
# plot mean
points = np.array(points)
plt.plot(range(N + 1), points.mean(axis=0)[:, 1], color="C1", label="EI")


plt.xlabel("Samples")
# reduce number of ticks
plt.xticks([0, 5, 10])
plt.ylim(-10, 0)
plt.yticks(np.linspace(-10, 0, 3))
plt.legend()
plt.savefig("concept_topk.png", dpi=300)

In [ ]:
N = 10
M = 25
plt.figure(figsize=(3.5, 3.5 / 1.2))
random_points = []
for i in range(M):
    point = run_experiment(bolift.AskTellFewShotMulti, N=N, aq="random")
    random_points.append(point)
    plt.plot(range(N + 1), [y for x, y in point], color="C0", alpha=0.1)
# plot mean
random_points = np.array(random_points)
plt.plot(range(N + 1), random_points.mean(axis=0)[:, 1], color="C0", label="Random")

greedy_points = []
for i in range(M):
    point = run_experiment(bolift.AskTellFewShotMulti, N=N, aq="greedy")
    greedy_points.append(point)
    plt.plot(range(N + 1), [y for x, y in point], color="C2", alpha=0.1)
# plot mean
greedy_points = np.array(greedy_points)
plt.plot(range(N + 1), greedy_points.mean(axis=0)[:, 1], color="C2", label="Greedy")

ei_points = []
for i in range(M):
    point = run_experiment(bolift.AskTellFewShotMulti, N=N, aq="expected_improvement")
    ei_points.append(point)
    plt.plot(range(N + 1), [y for x, y in point], color="C1", alpha=0.1)
# plot mean
ei_points = np.array(ei_points)
plt.plot(range(N + 1), ei_points.mean(axis=0)[:, 1], color="C1", label="EI")

plt.xlabel("Samples")
# reduce number of ticks
plt.xticks([0, 5, 10])
plt.ylim(-10, 0)
plt.yticks(np.linspace(-10, 0, 3))
plt.legend()
plt.savefig("concept_multi.png", dpi=300)